In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from typing import Optional, Any, List, Tuple, Dict
import math
from dataclasses import dataclass
import time

# Seeds
np.random.seed(42)
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device )

Using cpu


## NumPy Indexing in Multi-Dim: Implicit Rules

Consider a tensor:
```python
x.shape == (2, 3, 4) # [seq_len, n_heads, d_head]
````

Visual layout:

```
[
  [   # first axis-0 element (token 0)
    [x000, x001, x002, x003],   # head 0
    [x010, x011, x012, x013],   # head 1
    [x020, x021, x022, x023]    # head 2
  ],

  [   # second axis-0 element (token 1)
    [x100, x101, x102, x103],   # head 0
    [x110, x111, x112, x113],   # head 1
    [x120, x121, x122, x123]    # head 2
  ]
]
```

---

### 1. Missing slices are implied

If fewer slices are provided than the number of dimensions, NumPy assumes full slices (`:`) for the remaining axes.

```python
x[:, 1]        # (2, 4), equivalent to x[:, 1, :]
```

Result:

```
[
  [x010, x011, x012, x013],   # token 0, head 1
  [x110, x111, x112, x113]    # token 1, head 1
]
```

```python
x[0]          # (3, 4), equivalent to x[0, :, :]
```

Result:

```
[
  [x000, x001, x002, x003],   # head 0
  [x010, x011, x012, x013],   # head 1
  [x020, x021, x022, x023]    # head 2
]
```

---

### 2. Integer indices remove dimensions

An integer index selects a single element along that axis and drops the dimension.

```python
x[:, :, 2]    # (2, 3), axis 2 removed
```

Result:

```
[
  [x002, x012, x022],   # token 0, all heads
  [x102, x112, x122]    # token 1, all heads
]
```

---

### 3. Slice (`:`) keeps dimensions

Using a slice preserves the axis length.

```python
x[:, 1:2, :].shape   # (2, 1, 4)
```

Result (note the extra axis of length 1):

```
[
  [
    [x010, x011, x012, x013]   # head 1 kept as axis length 1
  ],
  [
    [x110, x111, x112, x113]
  ]
]
```

---

### 4. Trailing axes can be omitted

If only the first axes are specified, NumPy assumes `:` for all remaining axes.

```python
x[0]       # (3, 4), equivalent to x[0, :, :]
```

Result (same as above for token 0):

```
[
  [x000, x001, x002, x003],
  [x010, x011, x012, x013],
  [x020, x021, x022, x023]
]
```

---

### 5. Ellipsis (`...`)

Ellipsis expands to the required number of `:` to cover missing dimensions.

```python
x[..., 2]   # (2, 3), equivalent to x[:, :, 2]
```

Result:

```
[
  [x002, x012, x022],   # token 0
  [x102, x112, x122]    # token 1
]
```

```python
x[0, ...]   # (3, 4), equivalent to x[0, :, :]
```

Result:

```
[
  [x000, x001, x002, x003],
  [x010, x011, x012, x013],
  [x020, x021, x022, x023]
]
```

# Vanilla Bidirectional Attention

In [4]:
class BidirectionalAttention:
  """Bidirectional attention without causal mask"""

  def __init__(self, d_model:int, n_heads:int):
    self.d_model = d_model
    self.n_heads = n_heads
    self.d_head = d_model//n_heads

    # Weight matrices
    self.W_Q = np.random.randn(d_model, d_model) * 0.02
    self.W_K = np.random.randn(d_model, d_model) * 0.02
    self.W_V = np.random.randn(d_model, d_model) * 0.02
    self.W_O = np.random.randn(d_model, d_model) * 0.02

    print(f"Bidirectional attention: d_model={d_model}, n_heads={n_heads}, d_head={self.d_head}")

  def forward(self, x:np.ndarray ) -> Tuple[np.ndarray, np.ndarray]:
    """
    x: [seq_len, d_model]
    Returns: sequence output [seq_len, d_model], attention [seq_len, seq_len]
    """

    seq_len = x.shape[0]

    # Compute Q, K, V
    Q = x @ self.W_Q # [seq_len, d_model]
    K = x @ self.K_Q # [seq_len, d_model]
    V = x @ self.V_Q # [seq_len, d_model]
    print(f"\nQ,K,V shapes: {Q.shape}")

    # Reshape for the multi-head, needs [L, h, d_k]
    Q = Q.reshape(seq_len, self.n_heads, self.d_head)  #  n_heads * d_head = d_model
    K = K.reshape(seq_len, self.n_heads, self.d_head)
    V = V.reshape(seq_len, self.n_heads, self.d_head)

    # Compute attention without causal masking
    scores = np.zeros(self.n_heads, seq_len, seq_len)

    for head in range(self.n_heads):
      QK = Q[:, head] @ K[:,head].T # [L, d_k] @ [d_k, L] = [L, L]
